# Chapter 4. Sequence annotation objects

Chapter 3 introduced the sequence classes. Immediately “above” the Seq class is the Sequence Record or SeqRecord class, defined in the Bio.SeqRecord module. This class allows higher level features such as identifiers and features (as SeqFeature objects) to be associated with the sequence, and is used throughout the sequence input/output interface Bio.SeqIO described fully in Chapter 5.

While this chapter should cover most things to do with the SeqRecord and SeqFeature objects in this chapter, you may also want to read the SeqRecord wiki page (http://biopython.org/wiki/SeqRecord), and the built in documentation (also online – SeqRecord and SeqFeature):
```
from Bio.SeqRecord import SeqRecord
help(SeqRecord)
```


## 4.2 Creating a SeqRecord
### 4.2.1 SeqRecord objects from scratch

In [ ]:
from Bio.Seq import Seq
simple_seq = Seq("GATC")
from Bio.SeqRecord import SeqRecord
simple_seq_r = SeqRecord(simple_seq)

Additionally, you can also pass the id, name and description to the initialization function, but if not they will be set as strings indicating they are unknown, and can be modified subsequently:

In [ ]:
simple_seq_r.id
simple_seq_r.id = "AC12345"
simple_seq_r.description = "Made up sequence I wish I could write a paper about"
print(simple_seq_r.description)
simple_seq_r.seq

Made up sequence I wish I could write a paper about


Seq('GATC')

Including an identifier is very important if you want to output your SeqRecord to a file. You would normally include this when creating the object:

In [ ]:
from Bio.Seq import Seq
simple_seq = Seq("GATC")
from Bio.SeqRecord import SeqRecord
simple_seq_r = SeqRecord(simple_seq, id="AC12345")

As mentioned above, the SeqRecord has an dictionary attribute annotations. This is used for any miscellaneous annotations that doesn’t fit under one of the other more specific attributes. Adding annotations is easy, and just involves dealing directly with the annotation dictionary:

In [ ]:
simple_seq_r.annotations["evidence"] = "None. I just made it up."
print(simple_seq_r.annotations)
print(simple_seq_r.annotations["evidence"])

{'evidence': 'None. I just made it up.'}
None. I just made it up.


Working with per-letter-annotations is similar, letter_annotations is a dictionary like attribute which will let you assign any Python sequence (i.e. a string, list or tuple) which has the same length as the sequence:

In [ ]:
simple_seq_r.letter_annotations["phred_quality"] = [40, 40, 38, 30]
print(simple_seq_r.letter_annotations)
print(simple_seq_r.letter_annotations["phred_quality"])

{'phred_quality': [40, 40, 38, 30]}
[40, 40, 38, 30]


### 4.2.2 SeqRecord objects from FASTA files
This example uses a fairly large FASTA file containing the whole sequence for Yersinia pestis biovar Microtus
str. 91001 plasmid pPCP1, originally downloaded from the NCBI. This file is included with the Biopython
unit tests under the GenBank folder, or online [NC_005816.fna](https://raw.githubusercontent.com/biopython/biopython/master/Tests/GenBank/NC_005816.fna) from our website.
The file starts like this - and you can check there is only one record present (i.e. only one line starting
with a greater than symbol):
```
gi|45478711|ref|NC_005816.1| Yersinia pestis biovar Microtus ... pPCP1, complete sequence TGTAACGAACGGTGCAATAGTGATCCACACCCAACGCCTGAAATCAGATCCAGGGGGTAATCTGCTCTCC
```

In [ ]:
from Bio import SeqIO
record = SeqIO.read("NC_005816.fna", "fasta")
record

SeqRecord(seq=Seq('TGTAACGAACGGTGCAATAGTGATCCACACCCAACGCCTGAAATCAGATCCAGG...CTG'), id='gi|45478711|ref|NC_005816.1|', name='gi|45478711|ref|NC_005816.1|', description='gi|45478711|ref|NC_005816.1| Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, complete sequence', dbxrefs=[])

In [ ]:
record.seq

Seq('TGTAACGAACGGTGCAATAGTGATCCACACCCAACGCCTGAAATCAGATCCAGG...CTG')

In [ ]:
record.id
record.name
record.description

'gi|45478711|ref|NC_005816.1| Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, complete sequence'

In [ ]:
record.dbxrefs
record.annotations
record.letter_annotations
record.features

[]

### 4.2.3 SeqRecord objects from GenBank files
As in the previous example, we’re going to look at the whole sequence for Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, originally downloaded from the NCBI, but this time as a GenBank file. Again, this file is included with the Biopython unit tests under the GenBank folder, or online [NC_005816.gb](https://raw.githubusercontent.com/biopython/biopython/master/Tests/GenBank/NC_005816.gb) from our website.

## 4.3 Feature, location and position objects

### 4.3.2 Positions and locations

In [ ]:
from Bio import SeqFeature
start_pos = SeqFeature.AfterPosition(5)
end_pos = SeqFeature.BetweenPosition(9, left=8, right=9)
my_location = SeqFeature.SimpleLocation(start_pos, end_pos)

In [ ]:
print(my_location)

[>5:(8^9)]


In [ ]:
my_location.start
print(my_location.start)
my_location.end
print(my_location.end)

>5
(8^9)


In [ ]:
int(my_location.start)
int(my_location.end)

9

In [ ]:
exact_location = SeqFeature.SimpleLocation(5, 9)
print(exact_location)
exact_location.start
int(exact_location.start)

[5:9]


5

In [ ]:
from Bio import SeqIO
my_snp = 4350
record = SeqIO.read("NC_005816.gb", "genbank")
for feature in record.features:
    if my_snp in feature:
        print("%s %s" % (feature.type, feature.qualifiers.get("db_xref")))

source ['taxon:229193']
gene ['GeneID:2767712']
CDS ['GI:45478716', 'GeneID:2767712']


### 4.3.3 Sequence described by a feature or location

In [ ]:
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, SimpleLocation
seq = Seq("ACCGAGACGGCAAAGGCTAGCATAGGTATGAGACTTCCTTCCTGCCAGTGCTGAGGAACTGGGAGCCTAC")
feature = SeqFeature(SimpleLocation(5, 18, strand=-1), type="gene")

In [ ]:
feature_seq = seq[feature.location.start : feature.location.end].reverse_complement()
print(feature_seq)

AGCCTTTGCCGTC


In [ ]:
feature_seq = feature.extract(seq)
print(feature_seq)

AGCCTTTGCCGTC


In [ ]:
print(len(feature_seq))
print(len(feature))
print(len(feature.location))

13
13
13


## 4.4 Comparison

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
record1 = SeqRecord(Seq("ACGT"), id="test")
record2 = SeqRecord(Seq("ACGT"), id="test")

As of Biopython 1.67, SeqRecord comparison like `record1 == record2` will instead raise an explicit error to avoid people being caught out by this:

In [ ]:
record1 == record2  # NotImplementedError: SeqRecord comparison is deliberately not implemented. Explicitly compare the attributes of interest.

NotImplementedError: ignored

Instead you should check the attributes you are interested in, for example the identifier and the sequence:

In [ ]:
record1.id == record2.id
record1.seq == record2.seq

True

## 4.6 The format method

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
record = SeqRecord(
    Seq(
        "MMYQQGCFAGGTVLRLAKDLAENNRGARVLVVCSEITAVTFRGPSETHLDSMVGQALFGD"
        "GAGAVIVGSDPDLSVERPLYELVWTGATLLPDSEGAIDGHLREVGLTFHLLKDVPGLISK"
        "NIEKSLKEAFTPLGISDWNSTFWIAHPGGPAILDQVEAKLGLKEEKMRATREVLSEYGNM"
        "SSAC"
    ),
    id="gi|14150838|gb|AAK54648.1|AF376133_1",
    description="chalcone synthase [Cucumis sativus]",
)
print(record.format("fasta"))

>gi|14150838|gb|AAK54648.1|AF376133_1 chalcone synthase [Cucumis sativus]
MMYQQGCFAGGTVLRLAKDLAENNRGARVLVVCSEITAVTFRGPSETHLDSMVGQALFGD
GAGAVIVGSDPDLSVERPLYELVWTGATLLPDSEGAIDGHLREVGLTFHLLKDVPGLISK
NIEKSLKEAFTPLGISDWNSTFWIAHPGGPAILDQVEAKLGLKEEKMRATREVLSEYGNM
SSAC



## 4.7 Slicing a SeqRecord

In [ ]:
from Bio import SeqIO
record = SeqIO.read("NC_005816.gb", "genbank")
record

SeqRecord(seq=Seq('TGTAACGAACGGTGCAATAGTGATCCACACCCAACGCCTGAAATCAGATCCAGG...CTG'), id='NC_005816.1', name='NC_005816', description='Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, complete sequence', dbxrefs=['Project:58037'])

In [ ]:
len(record)
len(record.features)

41

In [ ]:
print(record.features[20])

type: gene
location: [4342:4780](+)
qualifiers:
    Key: db_xref, Value: ['GeneID:2767712']
    Key: gene, Value: ['pim']
    Key: locus_tag, Value: ['YP_pPCP05']



In [ ]:
print(record.features[21])

type: CDS
location: [4342:4780](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GI:45478716', 'GeneID:2767712']
    Key: gene, Value: ['pim']
    Key: locus_tag, Value: ['YP_pPCP05']
    Key: note, Value: ['similar to many previously sequenced pesticin immunity protein entries of Yersinia pestis plasmid pPCP, e.g. gi| 16082683|,ref|NP_395230.1| (NC_003132) , gi|1200166|emb|CAA90861.1| (Z54145 ) , gi|1488655| emb|CAA63439.1| (X92856) , gi|2996219|gb|AAC62543.1| (AF053945) , and gi|5763814|emb|CAB531 67.1| (AL109969)']
    Key: product, Value: ['pesticin immunity protein']
    Key: protein_id, Value: ['NP_995571.1']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['MGGGMISKLFCLALIFLSSSGLAEKNTYTAKDILQNLELNTFGNSLSHGIYGKQTTFKQTEFTNIKSNTKKHIALINKDNSWMISLKILGIKRDEYTVCFEDFSLIRPPTYVAIHPLLIKKVKSGNFIVVKEIKKSIPGCTVYYH']



In [ ]:
sub_record = record[4300:4800]
sub_record
len(sub_record)
len(sub_record.features)

2

In [ ]:
print(sub_record.features[0])

type: gene
location: [42:480](+)
qualifiers:
    Key: db_xref, Value: ['GeneID:2767712']
    Key: gene, Value: ['pim']
    Key: locus_tag, Value: ['YP_pPCP05']



In [ ]:
print(sub_record.features[1])

type: CDS
location: [42:480](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GI:45478716', 'GeneID:2767712']
    Key: gene, Value: ['pim']
    Key: locus_tag, Value: ['YP_pPCP05']
    Key: note, Value: ['similar to many previously sequenced pesticin immunity protein entries of Yersinia pestis plasmid pPCP, e.g. gi| 16082683|,ref|NP_395230.1| (NC_003132) , gi|1200166|emb|CAA90861.1| (Z54145 ) , gi|1488655| emb|CAA63439.1| (X92856) , gi|2996219|gb|AAC62543.1| (AF053945) , and gi|5763814|emb|CAB531 67.1| (AL109969)']
    Key: product, Value: ['pesticin immunity protein']
    Key: protein_id, Value: ['NP_995571.1']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['MGGGMISKLFCLALIFLSSSGLAEKNTYTAKDILQNLELNTFGNSLSHGIYGKQTTFKQTEFTNIKSNTKKHIALINKDNSWMISLKILGIKRDEYTVCFEDFSLIRPPTYVAIHPLLIKKVKSGNFIVVKEIKKSIPGCTVYYH']



In [ ]:
sub_record.annotations
sub_record.dbxrefs

[]

In [ ]:
sub_record.annotations["topology"] = "linear"

In [ ]:
sub_record.id
sub_record.name
sub_record.description

'Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, complete sequence'

In [ ]:
sub_record.description = "Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, partial"
print(sub_record.format("genbank")[:200] + "...")

LOCUS       NC_005816                500 bp    DNA     linear   UNK 01-JAN-1980
DEFINITION  Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, partial.
ACCESSION   NC_005816
VERSION     NC_0058...


## 4.8 Adding SeqRecord objects
For this example we’ll use some real data downloaded from the ENA sequence read archive, ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR020/SRR020192/SRR020192.fastq.gz (2MB) which unzips to a 19MB file `SRR020192.fastq`.
* To download test file :
```
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR020/SRR020192/SRR020192.fastq.gz
gzip -d SRR020192.fastq.gz
```

In [ ]:
from Bio import SeqIO
record = next(SeqIO.parse("SRR020192.fastq", "fastq"))
len(record)
print(record.seq)
print(record.letter_annotations["phred_quality"])

GATGACGGTGTCTACATTGTTCCCGACCACTCATCTCCTCTGTCATGCCCGAAACGTCTTCTCAAACCCGTCGT
[24, 23, 27, 30, 30, 30, 23, 23, 24, 23, 23, 30, 28, 27, 25, 25, 27, 27, 27, 22, 22, 24, 18, 18, 18, 30, 19, 19, 23, 23, 30, 30, 32, 32, 32, 30, 24, 23, 23, 27, 30, 32, 30, 32, 29, 28, 28, 17, 17, 17, 17, 24, 17, 17, 13, 15, 17, 25, 25, 24, 24, 23, 27, 27, 15, 15, 15, 15, 15, 17, 17, 11, 15, 15]


In [ ]:
left = record[:20]
print(left.seq)
print(left.letter_annotations["phred_quality"])
right = record[21:]
print(right.seq)
print(right.letter_annotations["phred_quality"])

GATGACGGTGTCTACATTGT
[24, 23, 27, 30, 30, 30, 23, 23, 24, 23, 23, 30, 28, 27, 25, 25, 27, 27, 27, 22]
CCCGACCACTCATCTCCTCTGTCATGCCCGAAACGTCTTCTCAAACCCGTCGT
[24, 18, 18, 18, 30, 19, 19, 23, 23, 30, 30, 32, 32, 32, 30, 24, 23, 23, 27, 30, 32, 30, 32, 29, 28, 28, 17, 17, 17, 17, 24, 17, 17, 13, 15, 17, 25, 25, 24, 24, 23, 27, 27, 15, 15, 15, 15, 15, 17, 17, 11, 15, 15]


In [ ]:
edited = left + right
len(edited)
print(edited.seq)
print(edited.letter_annotations["phred_quality"])

# edited = record[:20] + record[21:]

GATGACGGTGTCTACATTGTCCCGACCACTCATCTCCTCTGTCATGCCCGAAACGTCTTCTCAAACCCGTCGT
[24, 23, 27, 30, 30, 30, 23, 23, 24, 23, 23, 30, 28, 27, 25, 25, 27, 27, 27, 22, 24, 18, 18, 18, 30, 19, 19, 23, 23, 30, 30, 32, 32, 32, 30, 24, 23, 23, 27, 30, 32, 30, 32, 29, 28, 28, 17, 17, 17, 17, 24, 17, 17, 13, 15, 17, 25, 25, 24, 24, 23, 27, 27, 15, 15, 15, 15, 15, 17, 17, 11, 15, 15]


In [ ]:
from Bio import SeqIO
record = SeqIO.read("NC_005816.gb", "genbank")
record
len(record)
len(record.features)
record.dbxrefs
record.annotations.keys()

dict_keys(['molecule_type', 'topology', 'data_file_division', 'date', 'accessions', 'sequence_version', 'gi', 'keywords', 'source', 'organism', 'taxonomy', 'references', 'comment'])

In [ ]:
shifted = record[2000:] + record[:2000]
shifted
len(shifted)

9609

In [ ]:
len(shifted.features)
shifted.dbxrefs
shifted.annotations.keys()

dict_keys(['molecule_type'])

In [ ]:
shifted.dbxrefs = record.dbxrefs[:]
shifted.annotations = record.annotations.copy()
shifted.dbxrefs
shifted.annotations.keys()

dict_keys(['molecule_type', 'topology', 'data_file_division', 'date', 'accessions', 'sequence_version', 'gi', 'keywords', 'source', 'organism', 'taxonomy', 'references', 'comment'])

## 4.9 Reverse-complementing SeqRecord objects

In [ ]:
from Bio import SeqIO
record = SeqIO.read("NC_005816.gb", "genbank")
print("%s %i %i %i %i" % (record.id, len(record), len(record.features), len(record.dbxrefs), len(record.annotations)))

NC_005816.1 9609 41 1 13


In [ ]:
rc = record.reverse_complement(id="TESTING")
print("%s %i %i %i %i" % (rc.id, len(rc), len(rc.features), len(rc.dbxrefs), len(rc.annotations)))

TESTING 9609 41 0 0
